# Temat testów:
### Czy istnieje istotna statystycznie różnica w poprawności odpowiedzi w zależności od odpowiedzi na pytania dotyczące preferowanego typu grafiki i formy graficznej? (grupy: wszystkie grafiki, grafiki A, grafiki B, grafiki 1,2,3)

In [1]:
from scipy import stats
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df_answers = pd.read_csv(
    '/Users/martasolarz/Studies/Thesis/data/correctness.csv',
    usecols=['ID', 'Quest1_before', 'Quest2_before', '1a_corr', '1b_corr', '2a_corr', '2b_corr', '3a_corr', '3b_corr']
)

df_quest = pd.read_csv(
    '/Users/martasolarz/Studies/Thesis/data/survey.csv',
    usecols=['ID', 'Quest1_after', 'Quest2_after']
)

df_merge = df_answers.merge(df_quest, on='ID')
df_merge.head()

,ID,Quest1_before,Quest2_before,1a_corr,1b_corr,2a_corr,2b_corr,3a_corr,3b_corr,Quest1_after,Quest2_after
0,1,B,wykres,1,1,0,0,1,1,B,wykres
1,2,B,wykres,1,1,1,1,0,1,B,wykres
2,3,B,mapa,1,1,1,1,0,0,B,mapa
3,4,B,mapa,1,1,1,1,1,0,B,wykres
4,5,B,tabela,1,1,1,1,1,1,B,tabela


Funkcje:

In [51]:
def prepate_set(list_of_cols_popr, quest1_before, quest1_after, quest2_before, quest2_after):
    new_row_popr = list_of_cols_popr[0]
    for i in list_of_cols_popr[1:]:
        new_row_popr = pd.concat([new_row_popr, i], ignore_index=True)

    new_row_quest1_b = quest1_before[0]
    for i in quest1_before[1:]:
        new_row_quest1_b = pd.concat([new_row_quest1_b, i], ignore_index=True)
        
    new_row_quest2_b = quest2_before[0]
    for i in quest2_before[1:]:
        new_row_quest2_b = pd.concat([new_row_quest2_b, i], ignore_index=True)
       
    new_row_quest1_a = quest1_after[0]
    for i in quest1_after[1:]:
        new_row_quest1_a = pd.concat([new_row_quest1_a, i], ignore_index=True)
        
    new_row_quest2_a = quest2_after[0]
    for i in quest2_after[1:]:
        new_row_quest2_a = pd.concat([new_row_quest2_a, i], ignore_index=True) 
        
    return pd.DataFrame({'poprawnosc': new_row_popr, 'quest1_before': new_row_quest1_b, 
                         'quest2_before': new_row_quest2_b, 'quest1_after': new_row_quest1_a,
                         'quest2_after': new_row_quest2_a})

def prepate_set_changes(list_of_cols_popr, quest1, quest2):
    new_row_popr = list_of_cols_popr[0]
    for i in list_of_cols_popr[1:]:
        new_row_popr = pd.concat([new_row_popr, i], ignore_index=True)

    new_row_quest1 = quest1[0]
    for i in quest1[1:]:
        new_row_quest1 = pd.concat([new_row_quest1, i], ignore_index=True)
        
    new_row_quest2 = quest2[0]
    for i in quest2[1:]:
        new_row_quest2 = pd.concat([new_row_quest2, i], ignore_index=True)

        
    return pd.DataFrame({'poprawnosc': new_row_popr, 'quest1': new_row_quest1, 
                         'quest2': new_row_quest2})

In [4]:
def create_contingency_table(df, col):
    return pd.crosstab(df['poprawnosc'], df[col])

In [5]:
def make_decision(p_val, alpha):
    if p_val < alpha:
        print('Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1:')
    else:
        print('Brak podstaw do odrzucenia H0.')

In [6]:
def conduct_chi2test(contingency_table, alpha):
    chi2_stat, p_val, _, _ = stats.chi2_contingency(contingency_table)

    print('Wartość statystyki chi-kwadrat:', chi2_stat)
    print('P-value:', p_val)
    make_decision(p_val, alpha)

### TESTY

**H0:** Nie ma istotnej statystycznie zależności między poprawnością odpowiedzi a wybraną preferowaną formą (mapa vs mapa + inne elementy; mapa vs tabela vs wykres vs tekst).
**H1:** Istnieje istotna statystycznie zależność między poprawnością odpowiedzi a wybraną preferowaną formą (mapa vs mapa + inne elementy; mapa vs tabela vs wykres vs tekst).
**Poziom istotności:** $\alpha=0.05$

Mamy dwie zmienne kategoryczne (dyskretne):
- poprawność $ \text{{0,1}} $
- preferowane formy

Stąd wykonujemy test chi-kwadrat.

In [8]:
ALPHA = 0.05

a) wszystkie grafiki

In [9]:
quest1_before = [df_merge['Quest1_before'] for i in range(6)]
quest2_before = [df_merge['Quest2_before'] for i in range(6)]
quest1_after = [df_merge['Quest1_after'] for i in range(6)]
quest2_after = [df_merge['Quest2_after'] for i in range(6)]

popr = []
for col in df_answers.columns.tolist():
    if col.endswith('corr'):
        popr.append(df_answers[col])
        
df = prepate_set(popr, quest1_before, quest1_after, quest2_before, quest2_after)

df

,poprawnosc,quest1_before,quest2_before,quest1_after,quest2_after
0,1,B,wykres,B,wykres
1,1,B,wykres,B,wykres
2,1,B,mapa,B,mapa
3,1,B,mapa,B,wykres
4,1,B,tabela,B,tabela
...,...,...,...,...,...
241,1,B,mapa,B,wykres
242,0,B,mapa,B,wykres
243,0,B,mapa,B,tekst
244,0,B,mapa,B,tabela


In [10]:
# Mapa vs Inne formy - before
contingency_table_q1_b = create_contingency_table(df, 'quest1_before')
contingency_table_q1_b

quest1_before,A,B
poprawnosc,,
0,5,67
1,19,155


In [11]:
conduct_chi2test(contingency_table_q1_b, ALPHA)

Wartość statystyki chi-kwadrat: 0.5182647626074353
P-value: 0.4715828248989259
Brak podstaw do odrzucenia H0.


In [12]:
# Mapa vs Inne formy - after
contingency_table_q1_a = create_contingency_table(df, 'quest1_after')
contingency_table_q1_a

quest1_after,A,B
poprawnosc,,
0,27,45
1,39,135


In [13]:
conduct_chi2test(contingency_table_q1_a, ALPHA)

Wartość statystyki chi-kwadrat: 5.160716721815859
P-value: 0.023103361396515018
Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1:


In [14]:
# Mapa vs Wykres vs Tabela vs Tekst - before
contingency_table_q2_b = create_contingency_table(df, 'quest2_before')
contingency_table_q2_b

quest2_before,mapa,tabela,tekst,wykres
poprawnosc,,,,
0,41,12,3,16
1,85,30,15,44


In [15]:
conduct_chi2test(contingency_table_q1_b, ALPHA)

Wartość statystyki chi-kwadrat: 0.5182647626074353
P-value: 0.4715828248989259
Brak podstaw do odrzucenia H0.


In [16]:
# Mapa vs Wykres vs Tabela vs Tekst - after
contingency_table_q2_a = create_contingency_table(df, 'quest2_after')
contingency_table_q2_a

quest2_after,mapa,tabela,tekst,wykres
poprawnosc,,,,
0,41,6,3,22
1,73,30,3,68


In [17]:
conduct_chi2test(contingency_table_q1_b, ALPHA)

Wartość statystyki chi-kwadrat: 0.5182647626074353
P-value: 0.4715828248989259
Brak podstaw do odrzucenia H0.


b) grafiki A

In [18]:
quest1_before = [df_merge['Quest1_before'] for i in range(3)]
quest2_before = [df_merge['Quest2_before'] for i in range(3)]
quest1_after = [df_merge['Quest1_after'] for i in range(3)]
quest2_after = [df_merge['Quest2_after'] for i in range(3)]

popr = [df_answers['1a_corr'], df_answers['2a_corr'], df_answers['3a_corr']]
        
df = prepate_set(popr, quest1_before, quest1_after, quest2_before, quest2_after)
df

,poprawnosc,quest1_before,quest2_before,quest1_after,quest2_after
0,1,B,wykres,B,wykres
1,1,B,wykres,B,wykres
2,1,B,mapa,B,mapa
3,1,B,mapa,B,wykres
4,1,B,tabela,B,tabela
...,...,...,...,...,...
118,0,B,mapa,B,wykres
119,1,B,mapa,B,wykres
120,0,B,mapa,B,tekst
121,1,B,mapa,B,tabela


In [19]:
# Mapa vs Inne formy - before
contingency_table_q1_b = create_contingency_table(df, 'quest1_before')
contingency_table_q1_b

quest1_before,A,B
poprawnosc,,
0,2,34
1,10,77


In [20]:
conduct_chi2test(contingency_table_q1_b, ALPHA)

Wartość statystyki chi-kwadrat: 0.4570017215491356
P-value: 0.49902856634830706
Brak podstaw do odrzucenia H0.


In [21]:
# Mapa vs Inne formy - after
contingency_table_q1_a = create_contingency_table(df, 'quest1_after')
contingency_table_q1_a

quest1_after,A,B
poprawnosc,,
0,12,24
1,21,66


In [22]:
conduct_chi2test(contingency_table_q1_a, ALPHA)

Wartość statystyki chi-kwadrat: 0.6783648264251712
P-value: 0.41015035950447143
Brak podstaw do odrzucenia H0.


In [23]:
# Mapa vs Wykres vs Tabela vs Tekst - before
contingency_table_q2_b = create_contingency_table(df, 'quest2_before')
contingency_table_q2_b

quest2_before,mapa,tabela,tekst,wykres
poprawnosc,,,,
0,20,6,1,9
1,43,15,8,21


In [24]:
conduct_chi2test(contingency_table_q1_b, ALPHA)

Wartość statystyki chi-kwadrat: 0.4570017215491356
P-value: 0.49902856634830706
Brak podstaw do odrzucenia H0.


In [25]:
# Mapa vs Wykres vs Tabela vs Tekst - after
contingency_table_q2_a = create_contingency_table(df, 'quest2_after')
contingency_table_q2_a

quest2_after,mapa,tabela,tekst,wykres
poprawnosc,,,,
0,21,3,2,10
1,36,15,1,35


In [26]:
conduct_chi2test(contingency_table_q2_a, ALPHA)

Wartość statystyki chi-kwadrat: 6.066150769644418
P-value: 0.10843582722801164
Brak podstaw do odrzucenia H0.


c) grafiki B

In [27]:
quest1_before = [df_merge['Quest1_before'] for i in range(3)]
quest2_before = [df_merge['Quest2_before'] for i in range(3)]
quest1_after = [df_merge['Quest1_after'] for i in range(3)]
quest2_after = [df_merge['Quest2_after'] for i in range(3)]

popr = [df_answers['1b_corr'], df_answers['2b_corr'], df_answers['3b_corr']]
        
df = prepate_set(popr, quest1_before, quest1_after, quest2_before, quest2_after)
df

,poprawnosc,quest1_before,quest2_before,quest1_after,quest2_after
0,1,B,wykres,B,wykres
1,1,B,wykres,B,wykres
2,1,B,mapa,B,mapa
3,1,B,mapa,B,wykres
4,1,B,tabela,B,tabela
...,...,...,...,...,...
118,1,B,mapa,B,wykres
119,0,B,mapa,B,wykres
120,0,B,mapa,B,tekst
121,0,B,mapa,B,tabela


In [28]:
# Mapa vs Inne formy - before
contingency_table_q1_b = create_contingency_table(df, 'quest1_before')
contingency_table_q1_b

quest1_before,A,B
poprawnosc,,
0,3,33
1,9,78


In [29]:
conduct_chi2test(contingency_table_q1_b, ALPHA)

Wartość statystyki chi-kwadrat: 6.633788961375382e-05
P-value: 0.9935014556753952
Brak podstaw do odrzucenia H0.


In [30]:
# Mapa vs Inne formy - after
contingency_table_q1_a = create_contingency_table(df, 'quest1_after')
contingency_table_q1_a

quest1_after,A,B
poprawnosc,,
0,15,21
1,18,69


In [31]:
conduct_chi2test(contingency_table_q1_a, ALPHA)

Wartość statystyki chi-kwadrat: 4.689110211308487
P-value: 0.03035436213327259
Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1:


In [32]:
# Mapa vs Wykres vs Tabela vs Tekst - before
contingency_table_q2_b = create_contingency_table(df, 'quest2_before')
contingency_table_q2_b

quest2_before,mapa,tabela,tekst,wykres
poprawnosc,,,,
0,21,6,2,7
1,42,15,7,23


In [33]:
conduct_chi2test(contingency_table_q1_b, ALPHA)

Wartość statystyki chi-kwadrat: 6.633788961375382e-05
P-value: 0.9935014556753952
Brak podstaw do odrzucenia H0.


In [34]:
# Mapa vs Wykres vs Tabela vs Tekst - after
contingency_table_q2_a = create_contingency_table(df, 'quest2_after')
contingency_table_q2_a

quest2_after,mapa,tabela,tekst,wykres
poprawnosc,,,,
0,20,3,1,12
1,37,15,2,33


In [35]:
conduct_chi2test(contingency_table_q1_b, ALPHA)

Wartość statystyki chi-kwadrat: 6.633788961375382e-05
P-value: 0.9935014556753952
Brak podstaw do odrzucenia H0.


d) grafiki 1,2,3

-> 1

In [36]:
popr = []
for col in df_merge.columns.tolist():
    if col.startswith('1') and col.endswith('_corr'):
        popr.append(df_merge[col])
        
quest1_before = [df_merge['Quest1_before'] for i in range(2)]
quest2_before = [df_merge['Quest2_before'] for i in range(2)]
quest1_after = [df_merge['Quest1_after'] for i in range(2)]
quest2_after = [df_merge['Quest2_after'] for i in range(2)]
        
df = prepate_set(popr, quest1_before, quest1_after, quest2_before, quest2_after)
df

,poprawnosc,quest1_before,quest2_before,quest1_after,quest2_after
0,1,B,wykres,B,wykres
1,1,B,wykres,B,wykres
2,1,B,mapa,B,mapa
3,1,B,mapa,B,wykres
4,1,B,tabela,B,tabela
...,...,...,...,...,...
77,1,B,mapa,B,wykres
78,1,B,mapa,B,wykres
79,1,B,mapa,B,tekst
80,1,B,mapa,B,tabela


In [37]:
for i in ['quest1_before', 'quest1_after', 'quest2_before', 'quest2_after']:
    print('----------------')
    print(i)
    tab = create_contingency_table(df, i)
    conduct_chi2test(tab, ALPHA)

----------------
quest1_before
Wartość statystyki chi-kwadrat: 0.0
P-value: 1.0
Brak podstaw do odrzucenia H0.
----------------
quest1_after
Wartość statystyki chi-kwadrat: 7.935670045045044
P-value: 0.004846967594060261
Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1:
----------------
quest2_before
Wartość statystyki chi-kwadrat: 0.9788288288288286
P-value: 0.8063745671387934
Brak podstaw do odrzucenia H0.
----------------
quest2_after
Wartość statystyki chi-kwadrat: 6.159720246562351
P-value: 0.10409284868832222
Brak podstaw do odrzucenia H0.


-> 2

In [38]:
popr = []
for col in df_merge.columns.tolist():
    if col.startswith('2') and col.endswith('_corr'):
        popr.append(df_merge[col])
        
quest1_before = [df_merge['Quest1_before'] for i in range(2)]
quest2_before = [df_merge['Quest2_before'] for i in range(2)]
quest1_after = [df_merge['Quest1_after'] for i in range(2)]
quest2_after = [df_merge['Quest2_after'] for i in range(2)]
        
df = prepate_set(popr, quest1_before, quest1_after, quest2_before, quest2_after)
df

,poprawnosc,quest1_before,quest2_before,quest1_after,quest2_after
0,0,B,wykres,B,wykres
1,1,B,wykres,B,wykres
2,1,B,mapa,B,mapa
3,1,B,mapa,B,wykres
4,1,B,tabela,B,tabela
...,...,...,...,...,...
77,1,B,mapa,B,wykres
78,1,B,mapa,B,wykres
79,1,B,mapa,B,tekst
80,0,B,mapa,B,tabela


In [39]:
for i in ['quest1_before', 'quest1_after', 'quest2_before', 'quest2_after']:
    print('----------------')
    print(i)
    tab = create_contingency_table(df, i)
    conduct_chi2test(tab, ALPHA)

----------------
quest1_before
Wartość statystyki chi-kwadrat: 0.37981487382740153
P-value: 0.5377023293868424
Brak podstaw do odrzucenia H0.
----------------
quest1_after
Wartość statystyki chi-kwadrat: 0.033372412407047594
P-value: 0.8550481858414848
Brak podstaw do odrzucenia H0.
----------------
quest2_before
Wartość statystyki chi-kwadrat: 0.8905849738568967
P-value: 0.8276992569297563
Brak podstaw do odrzucenia H0.
----------------
quest2_after
Wartość statystyki chi-kwadrat: 1.0588579037867332
P-value: 0.7870140608514232
Brak podstaw do odrzucenia H0.


-> 3

In [40]:
popr = []
for col in df_merge.columns.tolist():
    if col.startswith('3') and col.endswith('_corr'):
        popr.append(df_merge[col])
        
quest1_before = [df_merge['Quest1_before'] for i in range(2)]
quest2_before = [df_merge['Quest2_before'] for i in range(2)]
quest1_after = [df_merge['Quest1_after'] for i in range(2)]
quest2_after = [df_merge['Quest2_after'] for i in range(2)]
        
df = prepate_set(popr, quest1_before, quest1_after, quest2_before, quest2_after)
df

,poprawnosc,quest1_before,quest2_before,quest1_after,quest2_after
0,1,B,wykres,B,wykres
1,0,B,wykres,B,wykres
2,0,B,mapa,B,mapa
3,1,B,mapa,B,wykres
4,1,B,tabela,B,tabela
...,...,...,...,...,...
77,1,B,mapa,B,wykres
78,0,B,mapa,B,wykres
79,0,B,mapa,B,tekst
80,0,B,mapa,B,tabela


In [41]:
for i in ['quest1_before', 'quest1_after', 'quest2_before', 'quest2_after']:
    print('----------------')
    print(i)
    tab = create_contingency_table(df, i)
    conduct_chi2test(tab, ALPHA)

----------------
quest1_before
Wartość statystyki chi-kwadrat: 0.1385135135135135
P-value: 0.7097632756475021
Brak podstaw do odrzucenia H0.
----------------
quest1_after
Wartość statystyki chi-kwadrat: 1.553030303030303
P-value: 0.21268864307211574
Brak podstaw do odrzucenia H0.
----------------
quest2_before
Wartość statystyki chi-kwadrat: 3.533333333333333
P-value: 0.31646456315918847
Brak podstaw do odrzucenia H0.
----------------
quest2_after
Wartość statystyki chi-kwadrat: 12.322807017543859
P-value: 0.006355332891152612
Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1:


e) zadania 1a, 1b, 2a, 2b, 3a, 3b

In [42]:
df1a = df_merge[['Quest1_before', 'Quest1_after', 'Quest2_before', 'Quest2_after', '1a_corr']].rename(columns={'1a_corr':'poprawnosc'})
df2a = df_merge[['Quest1_before', 'Quest1_after', 'Quest2_before', 'Quest2_after', '2a_corr']].rename(columns={'2a_corr':'poprawnosc'})
df3a = df_merge[['Quest1_before', 'Quest1_after', 'Quest2_before', 'Quest2_after', '3a_corr']].rename(columns={'3a_corr':'poprawnosc'})

df1b = df_merge[['Quest1_before', 'Quest1_after', 'Quest2_before', 'Quest2_after', '1b_corr']].rename(columns={'1b_corr':'poprawnosc'})
df2b = df_merge[['Quest1_before', 'Quest1_after', 'Quest2_before', 'Quest2_after', '2b_corr']].rename(columns={'2b_corr':'poprawnosc'})
df3b = df_merge[['Quest1_before', 'Quest1_after', 'Quest2_before', 'Quest2_after', '3b_corr']].rename(columns={'3b_corr':'poprawnosc'})

dfs = [df1a, df1b, df2a, df2b, df3a, df3b]

In [43]:
for nr, dfx in enumerate(dfs):
    print('---------------------------------------------')
    print(nr+1)
    for i in ['Quest1_before', 'Quest1_after', 'Quest2_before', 'Quest2_after']:
        print('----------------')
        print(i)
        tab = create_contingency_table(dfx, i)
        conduct_chi2test(tab, ALPHA)

---------------------------------------------
1
----------------
Quest1_before
Wartość statystyki chi-kwadrat: 0.00038475975975976035
P-value: 0.9843502633116243
Brak podstaw do odrzucenia H0.
----------------
Quest1_after
Wartość statystyki chi-kwadrat: 5.406098484848484
P-value: 0.020066516230538914
Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1:
----------------
Quest2_before
Wartość statystyki chi-kwadrat: 0.5759523809523808
P-value: 0.9019147738707174
Brak podstaw do odrzucenia H0.
----------------
Quest2_after
Wartość statystyki chi-kwadrat: 6.593567251461989
P-value: 0.08604458771266413
Brak podstaw do odrzucenia H0.
---------------------------------------------
2
----------------
Quest1_before
Wartość statystyki chi-kwadrat: 0.0
P-value: 1.0
Brak podstaw do odrzucenia H0.
----------------
Quest1_after
Wartość statystyki chi-kwadrat: 0.8852272727272728
P-value: 0.34677395734420646
Brak podstaw do odrzucenia H0.
----------------
Quest2_before
Wartość statystyki chi-k

### Podsumowanie:
Istnieje istotna statystycznie zależność między poprawnością odpowiedzi a odpowiedzią na pytanie dotyczące preferowanej formy graficznej na poziomie istotności $\alpha=0.05$ dla:
- wszystkie grafiki, quest1_after
- grafiki B, quest1_after
- grafiki 1, quest1_after
- grafiki 3, quest2_after
- grafika 1a, quest1_after
- grafika 3b, quest2_after



### Analiza ossoby które zmieniły zdanie vs osoby które pozostały przy swojej opinii
**H0:** Nie ma istotnej statystycznie zależności między poprawnością odpowiedzi a zmianą zdania w pytaniu 1/2.
**H1:** Istnieje istotna statystycznie zależność między poprawnością odpowiedzi a zmianą zdania w pytaniu 1/2.
**Poziom istotności:** $\alpha=0.05$


In [49]:
df_merge['is_change_1'] = df_merge.apply(lambda row: row['Quest1_before'] != row['Quest1_after'], axis=1)
df_merge['is_change_2'] = df_merge.apply(lambda row: row['Quest2_before'] != row['Quest2_after'], axis=1)

df_merge

,ID,Quest1_before,Quest2_before,1a_corr,1b_corr,2a_corr,2b_corr,3a_corr,3b_corr,Quest1_after,Quest2_after,is_change_1,is_change_2
0,1,B,wykres,1,1,0,0,1,1,B,wykres,False,False
1,2,B,wykres,1,1,1,1,0,1,B,wykres,False,False
2,3,B,mapa,1,1,1,1,0,0,B,mapa,False,False
3,4,B,mapa,1,1,1,1,1,0,B,wykres,False,True
4,5,B,tabela,1,1,1,1,1,1,B,tabela,False,False
5,6,B,wykres,1,1,1,1,0,1,B,mapa,False,True
6,7,B,tekst,1,1,1,1,1,1,B,wykres,False,True
7,11,B,mapa,1,0,0,0,0,1,B,wykres,False,True
8,12,B,mapa,1,1,0,0,0,0,B,mapa,False,False
9,13,A,mapa,1,1,1,1,1,0,A,mapa,False,False


a) wszystkie grafiki

In [52]:
# wszystkie
popr = []
for col in df_answers.columns.tolist():
    if col.endswith('corr'):
        popr.append(df_answers[col])
        
changes1 = [df_merge['is_change_1'] for i in range(6)]
changes2 = [df_merge['is_change_2'] for i in range(6)]
df = prepate_set_changes(popr, changes1, changes2)
df

,poprawnosc,quest1,quest2
0,1,False,False
1,1,False,False
2,1,False,False
3,1,False,True
4,1,False,False
...,...,...,...
241,1,False,True
242,0,False,True
243,0,False,True
244,0,False,True


In [55]:
for i in ['quest1', 'quest2']:
    print('----------------')
    print(i)
    tab = create_contingency_table(df, i)
    conduct_chi2test(tab, ALPHA)

----------------
quest1
Wartość statystyki chi-kwadrat: 3.717360196493188
P-value: 0.05384944288623782
Brak podstaw do odrzucenia H0.
----------------
quest2
Wartość statystyki chi-kwadrat: 0.21060853687739414
P-value: 0.6462908566505332
Brak podstaw do odrzucenia H0.


In [57]:
# grafiki A
popr = []
for col in df_answers.columns.tolist():
    if col.endswith('a_corr'):
        popr.append(df_answers[col])
        
changes1 = [df_merge['is_change_1'] for i in range(3)]
changes2 = [df_merge['is_change_2'] for i in range(3)]
df = prepate_set_changes(popr, changes1, changes2)
df

,poprawnosc,quest1,quest2
0,1,False,False
1,1,False,False
2,1,False,False
3,1,False,True
4,1,False,False
...,...,...,...
118,0,False,True
119,1,False,True
120,0,False,True
121,1,False,True


In [58]:
for i in ['quest1', 'quest2']:
    print('----------------')
    print(i)
    tab = create_contingency_table(df, i)
    conduct_chi2test(tab, ALPHA)

----------------
quest1
Wartość statystyki chi-kwadrat: 0.5850237135483183
P-value: 0.4443498952350563
Brak podstaw do odrzucenia H0.
----------------
quest2
Wartość statystyki chi-kwadrat: 0.0
P-value: 1.0
Brak podstaw do odrzucenia H0.


In [59]:
# grafiki B
popr = []
for col in df_answers.columns.tolist():
    if col.endswith('b_corr'):
        popr.append(df_answers[col])
        
changes1 = [df_merge['is_change_1'] for i in range(3)]
changes2 = [df_merge['is_change_2'] for i in range(3)]
df = prepate_set_changes(popr, changes1, changes2)
df

,poprawnosc,quest1,quest2
0,1,False,False
1,1,False,False
2,1,False,False
3,1,False,True
4,1,False,False
...,...,...,...
118,1,False,True
119,0,False,True
120,0,False,True
121,0,False,True


In [60]:
for i in ['quest1', 'quest2']:
    print('----------------')
    print(i)
    tab = create_contingency_table(df, i)
    conduct_chi2test(tab, ALPHA)

----------------
quest1
Wartość statystyki chi-kwadrat: 2.966708739091102
P-value: 0.08499461498099148
Brak podstaw do odrzucenia H0.
----------------
quest2
Wartość statystyki chi-kwadrat: 0.3958866139846738
P-value: 0.5292212634953144
Brak podstaw do odrzucenia H0.


In [61]:
# grafiki 1,2,3
popr = []
for col in df_merge.columns.tolist():
    if col.startswith('1') and col.endswith('_corr'):
        popr.append(df_merge[col])

changes1 = [df_merge['is_change_1'] for i in range(2)]
changes2 = [df_merge['is_change_2'] for i in range(2)]
df1 = prepate_set_changes(popr, changes1, changes2)

popr = []
for col in df_merge.columns.tolist():
    if col.startswith('2') and col.endswith('_corr'):
        popr.append(df_merge[col])

changes1 = [df_merge['is_change_1'] for i in range(2)]
changes2 = [df_merge['is_change_2'] for i in range(2)]
df2 = prepate_set_changes(popr, changes1, changes2)

popr = []
for col in df_merge.columns.tolist():
    if col.startswith('3') and col.endswith('_corr'):
        popr.append(df_merge[col])

changes1 = [df_merge['is_change_1'] for i in range(2)]
changes2 = [df_merge['is_change_2'] for i in range(2)]
df3 = prepate_set_changes(popr, changes1, changes2)
df3

,poprawnosc,quest1,quest2
0,1,False,False
1,0,False,False
2,0,False,False
3,1,False,True
4,1,False,False
...,...,...,...
77,1,False,True
78,0,False,True
79,0,False,True
80,0,False,True


In [62]:
for i in ['quest1', 'quest2']:
    print('----------------')
    print(i)
    tab = create_contingency_table(df1, i)
    conduct_chi2test(tab, ALPHA)

----------------
quest1
Wartość statystyki chi-kwadrat: 6.087019636824325
P-value: 0.013617860689217585
Istnieją podstawy do odrzucenia H0, przyjmujemy hipotezę H1:
----------------
quest2
Wartość statystyki chi-kwadrat: 0.0
P-value: 1.0
Brak podstaw do odrzucenia H0.


In [68]:
# w pytaniu 1 wyszło H1: sprawdzamy dla 1a i 1b
df1a = df_merge[['is_change_1', '1a_corr']].rename(columns={'1a_corr':'poprawnosc', 'is_change_1': 'quest1'})
df1b = df_merge[['is_change_1', '1b_corr']].rename(columns={'1b_corr':'poprawnosc', 'is_change_1': 'quest1'})

tab = create_contingency_table(df1a, 'quest1')
print(conduct_chi2test(tab, ALPHA))

tab = create_contingency_table(df1b, 'quest1')
print(conduct_chi2test(tab, ALPHA))

Wartość statystyki chi-kwadrat: 2.6148967978395055
P-value: 0.10586441215755699
Brak podstaw do odrzucenia H0.
None
Wartość statystyki chi-kwadrat: 1.486362390350877
P-value: 0.22278171435089766
Brak podstaw do odrzucenia H0.
None


In [63]:
for i in ['quest1', 'quest2']:
    print('----------------')
    print(i)
    tab = create_contingency_table(df2, i)
    conduct_chi2test(tab, ALPHA)

----------------
quest1
Wartość statystyki chi-kwadrat: 0.07181000777859668
P-value: 0.7887194137435113
Brak podstaw do odrzucenia H0.
----------------
quest2
Wartość statystyki chi-kwadrat: 3.0676354089904216
P-value: 0.07986556189878524
Brak podstaw do odrzucenia H0.


In [69]:
for i in ['quest1', 'quest2']:
    print('----------------')
    print(i)
    tab = create_contingency_table(df3, i)
    conduct_chi2test(tab, ALPHA)

----------------
quest1
Wartość statystyki chi-kwadrat: 0.640625
P-value: 0.42348456621606534
Brak podstaw do odrzucenia H0.
----------------
quest2
Wartość statystyki chi-kwadrat: 0.46124999999999994
P-value: 0.4970403627842026
Brak podstaw do odrzucenia H0.


Podsumowanie:
Zmiana zdania nie wpłynęła na poprawność odpowiedzi w sposób istotny statystycznie poza grafikami typu 1.